# TiDB Serverless Vector Search

In [ ]:
# Install Sentence Transformer & Connector for TiDB
!pip install sentence-transformers
!pip install mysql-connector-python

## Use SentenceTransformers and convert sentences into text embedding

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
input_texts = [
    'Anteraja menyediakan tiga pilihan layanan, yaitu Regular, Next-Day dan Same-Day. Untuk Regular merupakan jasa pengiriman parsel dalam kurun waktu 2-4 hari (Wilayah Pulau Jawa) dan 5-9 hari (Wilayah Luar Pulau Jawa), kemudian untuk Next-Day merupakan jasa pengiriman parsel dalam kurun waktu 1 hari, sedangkan untuk Same-Day merupakan jasa pengiriman parsel dalam kurun waktu 8 jam',
    'Proses pick up parsel akan dimulai pada pukul 10:00 s.d pukul 20:00, sedangkan untuk proses pengiriman parsel dimulai pada pukul 08:00 s.d pukul 20:00 setiap harinya. Request pickup melebihi jam diatas berpotensi dikirim keesokan harinya.',
    'Jangkauan pengiriman Anteraja sudah mencakup 30 provinsi di Indonesia, untuk lebih lengkapnya silahkan masuk ke fitur cek tarif di Aplikasi Anteraja',
    'Kamu dapat melakukan cancel order dengan cara memilih pesanan yang ingin dibatalkan pada menu My Activity kemudian klik tombol cancel order. Pesanan hanya dapat dibatalkan sebelum paket berhasil terpickup.',
    'Batas pengiriman yang dapat diproses oleh ANTERAJA adalah 50 (lima puluh) kilogram, apabila melebihi batas berat yang telah ditentukan maka pihak ANTERAJA dapat menolak melakukan pengambilan atas transaksi terkait.']
text_embeddings = model.encode(input_texts)

### Want to see how is text embedding look like?

In [ ]:
print(text_embeddings)

In [ ]:
print("Size of array for each sentence is")
print(text_embeddings[1].size)

## Connect to TiDB Serverless & prepare the table

In [ ]:
# Vector Playground
import mysql.connector

conn = mysql.connector.connect(user='<your-tidb-user>', password='<your-tidb-password>', host='<your-tidb-host>', port=4000, database='test')
mycursor = conn.cursor()

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS my_text")
mycursor.execute("CREATE TABLE my_text (str TEXT, embeddings VECTOR)")

## Insert our text embedding to TiDB Serverless as Vector data

In [ ]:
sql = "INSERT INTO my_text (str, embeddings) VALUES (%s, %s)"

for sentence, embedding in zip(input_texts, text_embeddings):
  val = (sentence, str(embedding.tolist()))
  mycursor.execute(sql, val)
conn.commit()

## Compare our search text with embedding data

In [ ]:
search_text = "Kemana saja jangkauan pengiriman yang dapat dilakukan oleh Anteraja"
search_embedding = str(model.encode(search_text).tolist())

In [ ]:
search_text = "Bagaimana melakukan cancel order?"
search_embedding = str(model.encode(search_text).tolist())

In [ ]:
# Let see our search embedding
print(model.encode(search_text))

In [ ]:
sql_str = "SELECT a.str,vec_cosine_distance(a.embeddings,'" + search_embedding + "') as sim_score FROM my_text a ORDER BY sim_score ASC LIMIT 3"
mycursor.execute(sql_str)

myresult = mycursor.fetchall()
for x in myresult:
  print(x)

In [ ]:
# let reorder the list
min_tuple = min(myresult, key=lambda x: x[1])


In [ ]:
print("Q: " + search_text)
print("A: " + min_tuple[0])
print("Score: " + str(min_tuple[1]))

In [ ]:
conn.close()